In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  

In [2]:
import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

In [3]:
NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (100, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [4]:
col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization", "other"]

rows = []

In [5]:
norm=True

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[143.8888, 127.1705, 117.5357], std=[69.8313, 64.5137, 66.9933])])
else:
   # transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    transform=None

train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform, num_classes=2)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform, num_classes=2)

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform, num_classes=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
print(class_count)

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)
undersampler_smote = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * 50 * NUM_CLASSES_REDUCED), replacement=False)
weight *= class_count[0]

[5000   50]


In [7]:
train_loader = DataLoader(train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_smote_undersampled = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler_smote)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [8]:
# to be used in distance capped smote - get average tensor for the entire class 
dataset = train_loader_ratio.dataset
class0 = dataset.images[dataset.labels==0]
class1 = dataset.images[dataset.labels==1]
class0_avg = torch.mean(class0.float(), 0)
class1_avg = torch.mean(class1.float(), 0)
class_img_list = [class0, class1]
avg_tensors_list = [class0_avg, class1_avg]

In [ ]:
# triplet loss first few epochs + capped smote (cosine distance)

# 2 class triplet loss with capped SMOTE 


momentum=0
learning_rates = [(5e-6, 1e-2), (1e-6, 5e-3), (1e-6, 1e-2), (5e-6, 5e-2)]

cap_aucs = []
loss_fn_args = {}
loss_caps = [0.75, 1, 5]
torch.autograd.set_detect_anomaly(False)

loss_fn_args['distance'] = 'cosine'
start_epoch = 7

for cap in loss_caps:
    learning_rate_aucs = []
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10): 
            model_aucs = []
            embed_network = models.ConvNetOnlyEmbeddingsEarly(2)
            linear_probe = models.ConvNetLinearProbeEarly(2)
            complete_network = models.CompleteConvNet(embed_network, linear_probe)
            embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
            linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                _, train_losses = train.train_triplet_loss_smote(epoch, train_loader_tripletloss_smote, embed_network, embed_optimizer, verbose=False)
                print("Train loss: " + str(np.mean(np.array(train_losses))))
            for epoch in range(start_epoch, n_epochs+1):
                loss_fn_args['loss_cap'] = cap
                loss_fn_args['avg_tensors'] = []
                for k in range(2):
                    avg_tensor = embed_network(avg_tensors_list[k])
                    loss_fn_args['avg_tensors'].append(avg_tensor)
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, complete_network, linear_optimizer, verbose=False, loss_fn=loss_fns.CappedBCELossAvgDistance, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
                    model_aucs.append(auc) 
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)


    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    cap_aucs.append([auc_mean, auc_variance])



for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote_with_smote_triplet_loss", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm, "start_epoch=7"]
        rows.append(row)


Test set: Avg. loss: 0.0006935921907424927, AUC: 0.49950249999999996

Train loss: 2.177517574683876
Train loss: 1.6380580819336472
Train loss: 1.1406405074581218
Train loss: 0.7840143058710037


In [ ]:
df1 = pd.read_csv('results/convnet_early_tripletloss_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_early_tripletloss_aucs.csv', index=False)

rows = []

In [21]:
# cosine distance + capped loss using whole class average tensor
momentum=0
learning_rates = [1e-3, 5e-3, 5e-4, 1e-4]


cap_aucs=[]

    
loss_fn_args = {}
loss_caps = [0.75, 1, 5]
loss_fn_args['distance'] = 'cosine'

start_epoch = 2

learning_rate_aucs = []

for loss_cap in loss_caps:
    
    learning_rate_aucs = []
    
    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddingsEarly(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                loss_fn_args['avg_tensors'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False,loss_fn=loss_fns.CappedBCELossAvgDistance,loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                loss_fn_args['print_loss']= False
                loss_fn_args['avg_tensors'] = []
                for k in range(NUM_CLASSES_REDUCED):
                    _, avg_tensor = network(avg_tensors_list[k])
                    loss_fn_args['avg_tensors'].append(avg_tensor)
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELossAvgDistance, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
        
    cap_aucs.append([auc_mean, auc_variance])

for c in range(len(cap_aucs)):
    auc_mean = cap_aucs[c][0]
    auc_variance = cap_aucs[c][1]
    cap = loss_caps[c]
    for i in range(len(learning_rates)): 
        row = ["cosine_distance_capped_smote_avg", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm, None]
        rows.append(row)


Test set: Avg. loss: 0.001968830943107605, AUC: 0.6361665000000001


Test set: Avg. loss: 0.0006622534990310669, AUC: 0.7385760000000001


Test set: Avg. loss: 0.0007262906134128571, AUC: 0.6065165


Test set: Avg. loss: 0.0006622972488403321, AUC: 0.827749


Test set: Avg. loss: 0.0010347468256950377, AUC: 0.515918


Test set: Avg. loss: 0.0004892772287130356, AUC: 0.8685539999999999


Test set: Avg. loss: 0.0008309755921363831, AUC: 0.8625809999999999


Test set: Avg. loss: 0.0015312259197235107, AUC: 0.83128


Test set: Avg. loss: 0.0016856086254119873, AUC: 0.402399


Test set: Avg. loss: 0.0006069324612617493, AUC: 0.8425619999999999


Test set: Avg. loss: 0.0006350790858268738, AUC: 0.8384395


Test set: Avg. loss: 0.0005941447913646698, AUC: 0.8411694999999999


Test set: Avg. loss: 0.0008586450517177582, AUC: 0.620958


Test set: Avg. loss: 0.0005511795878410339, AUC: 0.866982


Test set: Avg. loss: 0.0007755221128463746, AUC: 0.880608


Test set: Avg. loss: 0.0004314573109149


Test set: Avg. loss: 0.002385180950164795, AUC: 0.4585135


Test set: Avg. loss: 0.0006397717595100403, AUC: 0.793819


Test set: Avg. loss: 0.0006296209692955017, AUC: 0.8043739999999999


Test set: Avg. loss: 0.0006140455603599549, AUC: 0.8322370000000001


Test set: Avg. loss: 0.0009520593583583832, AUC: 0.555158


Test set: Avg. loss: 0.000550226092338562, AUC: 0.825901


Test set: Avg. loss: 0.0005467916131019592, AUC: 0.83493


Test set: Avg. loss: 0.0005628498792648315, AUC: 0.8450339999999998


Test set: Avg. loss: 0.001990069031715393, AUC: 0.598636


Test set: Avg. loss: 0.0005644693374633789, AUC: 0.8142369999999999


Test set: Avg. loss: 0.0005276755094528198, AUC: 0.826524


Test set: Avg. loss: 0.0006087591052055359, AUC: 0.84587


Test set: Avg. loss: 0.0012775813341140748, AUC: 0.5571980000000001


Test set: Avg. loss: 0.000588628202676773, AUC: 0.819166


Test set: Avg. loss: 0.0006013973355293273, AUC: 0.831462


Test set: Avg. loss: 0.0005729582905769348, AUC: 0.844


Test set: Avg. loss: 0.007216182947158814, AUC: 0.628137


Test set: Avg. loss: 0.0005674745738506317, AUC: 0.8468640000000001


Test set: Avg. loss: 0.0005497297644615173, AUC: 0.856886


Test set: Avg. loss: 0.0008376042544841766, AUC: 0.8382499999999999


Test set: Avg. loss: 0.0030689326524734497, AUC: 0.47973


Test set: Avg. loss: 0.000624789983034134, AUC: 0.8018845000000001


Test set: Avg. loss: 0.0006059180498123169, AUC: 0.8152074999999999


Test set: Avg. loss: 0.0006095174252986908, AUC: 0.819117


Test set: Avg. loss: 0.0026719954013824463, AUC: 0.492266


Test set: Avg. loss: 0.0004913255572319031, AUC: 0.8649659999999999


Test set: Avg. loss: 0.00044897058606147765, AUC: 0.878911


Test set: Avg. loss: 0.0007488345801830292, AUC: 0.876372


Test set: Avg. loss: 0.0029769840240478516, AUC: 0.41818900000000003


Test set: Avg. loss: 0.0005388579368591309, AUC: 0.8791850000000001


Test set: Avg. loss: 0.0004893890768289566, AUC: 0.8862850000000001


Test set: Avg. loss:


Test set: Avg. loss: 0.0007637673318386077, AUC: 0.868422


Test set: Avg. loss: 0.004009966731071472, AUC: 0.35853350000000006


Test set: Avg. loss: 0.0006316071748733521, AUC: 0.8411200000000001


Test set: Avg. loss: 0.0006807064712047577, AUC: 0.8418865


Test set: Avg. loss: 0.000700340211391449, AUC: 0.852107


Test set: Avg. loss: 0.005758796453475952, AUC: 0.564535


Test set: Avg. loss: 0.0007850680351257324, AUC: 0.7025580000000001


Test set: Avg. loss: 0.0007985642552375793, AUC: 0.8287899999999999


Test set: Avg. loss: 0.00093643718957901, AUC: 0.788198


Test set: Avg. loss: 0.0022677001953125, AUC: 0.716523


Test set: Avg. loss: 0.0005763637721538544, AUC: 0.836851


Test set: Avg. loss: 0.0009074821174144744, AUC: 0.783124


Test set: Avg. loss: 0.0009487994909286499, AUC: 0.8463384999999999


Test set: Avg. loss: 0.0014523221850395202, AUC: 0.667135


Test set: Avg. loss: 0.0005363000035285949, AUC: 0.8213550000000001


Test set: Avg. loss: 0.0008451749682426453, A

In [22]:
df1 = pd.read_csv('results/convnet_early_tripletloss_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_early_tripletloss_aucs.csv', index=False)

rows = []

In [31]:
# triplet loss smote + no smote training 


momentum=0
learning_rates = [(5e-6, 5e-3), (1e-6, 1e-2), (5e-6, 1e-3), (1e-6, 5e-4)]

loss_fn_args = {}
torch.autograd.set_detect_anomaly(False)


start_epoch = 10


learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddingsEarly(2)
        linear_probe = models.ConvNetLinearProbeEarly(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(start_epoch):
            _, train_losses = train.train_triplet_loss_smote(epoch, train_loader_tripletloss_smote, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
                model_aucs.append(auc) 
        for epoch in range(start_epoch, n_epochs+1):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, complete_network, linear_optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)




for i in range(len(learning_rates)): 
    row = ["triplet_loss_first_stage_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, "start_epoch=10"]
    rows.append(row)


Test set: Avg. loss: 0.000691878616809845, AUC: 0.578823

Train loss: 2.1152768104699007
Train loss: 1.362831683675195
Train loss: 1.7595135125384969
Train loss: 1.1299688204838212
Train loss: 0.6808489777479961
Train loss: 0.6694605194838943
Train loss: 1.0802323112062588
Train loss: 0.6371187286771787
Train loss: 0.5292761120826576
Train loss: 0.6434453463858101

Test set: Avg. loss: 0.0006957791149616242, AUC: 0.3901265


Test set: Avg. loss: 0.002493227958679199, AUC: 0.686156


Test set: Avg. loss: 0.002454917073249817, AUC: 0.7115910000000001


Test set: Avg. loss: 0.0006940029859542847, AUC: 0.575422

Train loss: 2.1154672890711743
Train loss: 2.5143205014763366
Train loss: 1.750417664172543
Train loss: 0.8689699283071385
Train loss: 0.8691085994623269
Train loss: 0.5269735884514584
Train loss: 0.8888260748735659
Train loss: 0.8102662460819171
Train loss: 0.4519774074767046
Train loss: 0.6589806945460617

Test set: Avg. loss: 0.0006968896687030793, AUC: 0.42876899999999996


Te


Test set: Avg. loss: 0.0024006248712539675, AUC: 0.706141


Test set: Avg. loss: 0.002390863060951233, AUC: 0.71977


Test set: Avg. loss: 0.0006933940351009369, AUC: 0.5733465

Train loss: 1.2468103872742622
Train loss: 0.9911186125627749
Train loss: 0.708361944954866
Train loss: 0.9297731590878432
Train loss: 0.7769595031525679
Train loss: 0.9345260402958864
Train loss: 0.561731690434134
Train loss: 0.7298914440877878
Train loss: 0.6808384114010319
Train loss: 0.6182690114732001

Test set: Avg. loss: 0.0006925439834594726, AUC: 0.601132


Test set: Avg. loss: 0.0025254011154174803, AUC: 0.685949


Test set: Avg. loss: 0.00238752281665802, AUC: 0.7134469999999999


Test set: Avg. loss: 0.0006937408149242402, AUC: 0.49135149999999994

Train loss: 3.282004968755564
Train loss: 2.7901779394240895
Train loss: 3.0127792760824703
Train loss: 2.410826233162242
Train loss: 2.25373038137035
Train loss: 2.4250421983421226
Train loss: 2.363741488593399
Train loss: 2.4345395697909558
Train loss:

Train loss: 1.1333944349531915
Train loss: 0.4936956789842836
Train loss: 0.6847197781702515
Train loss: 0.6981978735346703
Train loss: 0.3060620749831959
Train loss: 0.5588422664411509
Train loss: 0.36043294410037385

Test set: Avg. loss: 0.0006749890446662903, AUC: 0.772106


Test set: Avg. loss: 0.001904020369052887, AUC: 0.3095


Test set: Avg. loss: 0.0025695160627365113, AUC: 0.533582


Test set: Avg. loss: 0.0006949560046195984, AUC: 0.4225094999999999

Train loss: 1.2430606727387494
Train loss: 1.7227268856801805
Train loss: 1.6863460939401274
Train loss: 2.156649841244813
Train loss: 1.2017113387964333
Train loss: 1.0084415826068562
Train loss: 1.0128434061244795
Train loss: 1.0255884934382833
Train loss: 1.2540636305596418
Train loss: 1.2080791615376807

Test set: Avg. loss: 0.0006946629583835602, AUC: 0.43576600000000004


Test set: Avg. loss: 0.001171343982219696, AUC: 0.5162720000000001


Test set: Avg. loss: 0.002005854547023773, AUC: 0.579844


Test set: Avg. loss: 0.000

NameError: name 'cap' is not defined

In [15]:
learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

In [ ]:
for i in range(len(learning_rates)): 
    row = ["triplet_loss_first_stage_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], cap, norm, "start_epoch=10"]
    rows.append(row)

In [32]:

for i in range(len(learning_rates)): 
    row = ["triplet_loss_first_stage_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, "start_epoch=10"]
    rows.append(row)

In [33]:
df1 = pd.read_csv('results/convnet_early_tripletloss_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_early_tripletloss_aucs.csv', index=False)

rows = []

In [36]:
# 2 class triplet loss no ratio 
# no smote 


momentum=0
learning_rates = [(5e-6, 1e-2), (1e-6, 5e-3), (1e-6, 1e-2), (5e-6, 5e-2)]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddingsEarly(2)
        linear_probe = models.ConvNetLinearProbeEarly(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss(epoch, train_loader_tripletloss, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)
    
learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)
    
    



for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm, None]
    rows.append(row)


Test set: Avg. loss: 0.0006957003772258758, AUC: 0.39529899999999996

Train loss: 2.310011562648093
Train loss: 2.2350221542036457
Train loss: 1.7336593600595074
Train loss: 1.3570892761467368
Train loss: 1.7184112235239357
Train loss: 1.1358157290015252
Train loss: 1.0679599012538885
Train loss: 0.921282701431566
Train loss: 1.1848033970328653
Train loss: 0.9810460426245525
Train loss: 0.9329271555706194
Train loss: 0.9048631927769655
Train loss: 0.5647244722979843
Train loss: 0.5945013852635767
Train loss: 0.5329235454273832

Test set: Avg. loss: 0.00039782971143722536, AUC: 0.918406


Test set: Avg. loss: 0.0004035674780607224, AUC: 0.927185


Test set: Avg. loss: 0.00037781354784965514, AUC: 0.935988


Test set: Avg. loss: 0.0006927546858787537, AUC: 0.530769

Train loss: 2.022161154610336
Train loss: 1.288486762031628
Train loss: 1.399418694957806
Train loss: 1.0709760223224665
Train loss: 0.8879150535650314
Train loss: 0.994375331386639
Train loss: 0.9763853713205666
Train loss:

Train loss: 1.970422636551462
Train loss: 1.4079155132269403
Train loss: 1.4688413837912735
Train loss: 1.4076758281440491
Train loss: 1.1350592856953858
Train loss: 1.8857001426872935
Train loss: 1.4556400149491182

Test set: Avg. loss: 0.00047363899648189545, AUC: 0.8819830000000001


Test set: Avg. loss: 0.00042409318685531617, AUC: 0.908577


Test set: Avg. loss: 0.0003766205757856369, AUC: 0.922854


Test set: Avg. loss: 0.0006900011897087098, AUC: 0.654073

Train loss: 1.9343711790765168
Train loss: 1.4965036446881141
Train loss: 1.744480819459174
Train loss: 1.445263187217105
Train loss: 1.3542424216391935
Train loss: 1.3084997363910553
Train loss: 1.4788874330793975
Train loss: 1.3785485469611587
Train loss: 1.6266570197548835
Train loss: 1.6530657175240244
Train loss: 1.117041161105891
Train loss: 1.1217445317347339
Train loss: 0.7305867747896037
Train loss: 1.335534641317501
Train loss: 1.165849501539947

Test set: Avg. loss: 0.0004729681611061096, AUC: 0.886551


Test set: A


Test set: Avg. loss: 0.00036962753534317016, AUC: 0.9312050000000001


Test set: Avg. loss: 0.0004138304591178894, AUC: 0.9353440000000002


Test set: Avg. loss: 0.000701473593711853, AUC: 0.40515

Train loss: 1.065467907744608
Train loss: 1.3622498276886668
Train loss: 1.5146699308589766
Train loss: 1.1519298059925152
Train loss: 1.2872650144965785
Train loss: 1.1337903233090783
Train loss: 1.0842262722884015
Train loss: 1.1602510191073083
Train loss: 0.7593976547763606
Train loss: 1.2732060164403005
Train loss: 1.3662886513266594
Train loss: 0.6023548021438015
Train loss: 1.124361228791012
Train loss: 1.1234989906572233
Train loss: 0.8597518346112245

Test set: Avg. loss: 0.00041623535752296446, AUC: 0.920745


Test set: Avg. loss: 0.0003963103443384171, AUC: 0.934286


Test set: Avg. loss: 0.00045862875878810883, AUC: 0.936785


Test set: Avg. loss: 0.0006941128373146058, AUC: 0.5132665

Train loss: 3.3644635339451443
Train loss: 2.1163280215232994
Train loss: 1.979751812044982
Tra

Train loss: 1.9346731878389978
Train loss: 1.5430769199019025
Train loss: 1.4607565668737812
Train loss: 1.393643997277424
Train loss: 1.3211874517665547
Train loss: 0.921329250381251
Train loss: 0.9669004671133248
Train loss: 0.9917047209800429
Train loss: 0.7282513793866345
Train loss: 1.0467327958459307
Train loss: 0.5617396865680719
Train loss: 0.876054488929214
Train loss: 0.6982500355714446

Test set: Avg. loss: 0.00039860232174396515, AUC: 0.9290079999999998


Test set: Avg. loss: 0.00047183363139629365, AUC: 0.937491


Test set: Avg. loss: 0.00032021838426589963, AUC: 0.948357


Test set: Avg. loss: 0.0006923727095127105, AUC: 0.5628065

Train loss: 1.5512303265796346
Train loss: 1.2353783249855042
Train loss: 1.1475682938174836
Train loss: 1.4969956195278533
Train loss: 0.979395692515525
Train loss: 1.053526867726806
Train loss: 0.7830445409580401
Train loss: 0.7794080646174728
Train loss: 0.9118682161258285
Train loss: 0.7145632490230973
Train loss: 0.9021045573198112
Train l

In [37]:
df1 = pd.read_csv('results/convnet_early_tripletloss_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_early_tripletloss_aucs.csv', index=False)

rows = []